
### extracting checkpoints, bronze, silver container URLs

In [0]:
%run "/Workspace/Users/schaudhuri1495@gmail.com/04. Common"

In [0]:
dbutils.widgets.text(name = "env", defaultValue = '', label = 'Enter the environment in lowercase')
env = dbutils.widgets.get("env")


#Reading data from Bronze table

In [0]:
def read_from_bronze_table(env):
    print('Reading the Bronze data table : ', end='')
    df_bronzeTraffic = (spark.readStream
                        .table(f"`databricks_{env}_ws`.`bronze`.raw_traffic"))
    
    print(f"Reading databricks_{env}_ws.bronze.raw_traffic Success!!")

    return(df_bronzeTraffic)


##handling duplicates rows

In [0]:
def remove_dups(df):
    print("Removing duplicate values: ", end='')
    df_dup = df.dropDuplicates()
    print("SUCCESS!!!!!!!!!!")
    return df_dup


## Handling NULL values by replacing them

In [0]:
def handle_null(df, columns):
    print('Replacing NULL vlaues on String columns with "unknown"', end='')
    df_string = df.fillna('Unknown',subset=columns)
    print('Success!!!!!!!!!!!!!!!!!!')


    print('Replacing NULL vlaues on Int columns with "0"', end='')
    df_clean = df_string.fillna(0,subset=columns)
    print('Success!!!!!!!!!!!!!!!!!!')


    return df_clean





## getting count of Electric vehicles by creating new column

In [0]:
def ev_count(df):
    print('Creating Electric Vehicle Count Column : ', end='')
    from pyspark.sql.functions import col
    df_ev = df.withColumn('Electric_Vehicle_Count', col('EV_Car') + col('EV_Bike'))

    print('Success!!!!!!!!!')
    return df_ev






#creating motor_vehicles_count() function

In [0]:
def motor_vehicles_count(df):
    print('CREATING motor_vehicles_count FUNCTION')
    from pyspark.sql.functions import col
    df_motor_vehicles_count = df.withColumn('motor_vehicles_count', col('Two_wheeled_motor_vehicles') + col('Cars_and_taxis') + col('Buses_and_coaches') + col('LGV_Type') + col('HGV_Type') + col('Electric_Vehicle_Count'))

    print("success!!!!!!!!!!!!!")

    return(df_motor_vehicles_count)


#creating a new column named as "Transformed_time" to print the current timestamp

In [0]:
def create_transformed_timestamp(df):
    print("Inside the create_transformed_timestamp Function")
    from pyspark.sql.functions import current_timestamp
    df_with_time = df.withColumn('Transformed_Time', current_timestamp()) 
    print("SUCCESS FROM create_transformed_timestamp FUNCTION")
    return(df_with_time)


##Writing the transformed data to Silver_Traffic table in Silver schema

In [0]:
def write_traffic_silverTable(StreamingDF, env):
    write_stream_silver = (StreamingDF.writeStream
                            .format('delta')
                            .option("checkpointLocation",checkpoints+'/SilverTrafficLoad/Checkpt')
                            .outputMode('append')
                            .queryName('SilverTrafficWriteStream')
                            .trigger(availableNow=True)
                            .toTable(f"`databricks_{env}_ws`.`silver`.`silver_traffic`"))

    write_stream_silver.awaitTermination()
    print(f"Writing databricks_{env}_ws.silver.silver_traffic SUCCESS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")


##calling the functions

In [0]:
#to fetch values from bronze table
df_trafficdata = read_from_bronze_table(env)

#to remove duplicate rows

df_dups = remove_dups(df_trafficdata)

#to replace any NULL values

columns = df_dups.schema.names
df_nulls = handle_null(df_dups, columns)


#to add a new column for EV only

df_ev = ev_count(df_nulls)


#adding a new column to get the count of all the motor vehicles

df_all_motor = motor_vehicles_count(df_ev)

#adding a new column named "Transformed_Time" to get the current timestamp

df_including_timestamp = create_transformed_timestamp(df_all_motor)


#writing to silver_traffic folder

write_traffic_silverTable(df_including_timestamp, env)

Reading the Bronze data table : Reading databricks_dev_ws.bronze.raw_traffic Success!!
Removing duplicate values: SUCCESS!!!!!!!!!!
Replacing NULL vlaues on String columns with "unknown"Success!!!!!!!!!!!!!!!!!!
Replacing NULL vlaues on Int columns with "0"Success!!!!!!!!!!!!!!!!!!
Creating Electric Vehicle Count Column : Success!!!!!!!!!
CREATING motor_vehicles_count FUNCTION
success!!!!!!!!!!!!!
Inside the create_transformed_timestamp Function
SUCCESS FROM create_transformed_timestamp FUNCTION
Writing databricks_dev_ws.silver.silver_traffic SUCCESS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [0]:
%sql
select count(*) from databricks_dev_ws.silver.silver_traffic;

count(1)
74184


In [0]:
%sql
select * from databricks_dev_ws.silver.silver_traffic where Record_ID  between '37080' and '37100' order by Record_ID;

Record_ID,Count_point_id,Direction_of_travel,Year,Count_date,hour,Region_id,Region_name,Local_authority_name,Road_name,Road_Category_ID,Start_junction_road_name,End_junction_road_name,Latitude,Longitude,Link_length_km,Pedal_cycles,Two_wheeled_motor_vehicles,Cars_and_taxis,Buses_and_coaches,LGV_Type,HGV_Type,EV_Car,EV_Bike,Extract_Time,Electric_Vehicle_Count,motor_vehicles_count,Transformed_Time
37080,58073,W,2014,5/8/2014 0:00,15,7,East of England,Cambridgeshire,A14,3,A1303,A1303,52.21335948,0.267274407,6.8,0,2,844,2,243,45,9,9,2024-07-15T18:09:56.809Z,18,1154,2024-07-15T20:36:04.239Z
37081,58073,W,2014,5/8/2014 0:00,16,7,East of England,Cambridgeshire,A14,3,A1303,A1303,52.21335948,0.267274407,6.8,0,6,880,4,264,40,7,7,2024-07-15T18:09:56.809Z,14,1208,2024-07-15T20:36:04.239Z
37082,58073,W,2014,5/8/2014 0:00,17,7,East of England,Cambridgeshire,A14,3,A1303,A1303,52.21335948,0.267274407,6.8,0,2,1016,1,170,23,2,2,2024-07-15T18:09:56.809Z,4,1216,2024-07-15T20:36:04.239Z
37083,58073,W,2014,5/8/2014 0:00,18,7,East of England,Cambridgeshire,A14,3,A1303,A1303,52.21335948,0.267274407,6.8,0,1,868,4,144,14,1,3,2024-07-15T18:09:56.809Z,4,1035,2024-07-15T20:36:04.239Z
37084,58073,W,2014,5/8/2014 0:00,7,7,East of England,Cambridgeshire,A14,3,A1303,A1303,52.21335948,0.267274407,6.8,0,12,2275,0,523,44,7,11,2024-07-15T18:09:56.809Z,18,2872,2024-07-15T20:36:04.239Z
37085,58073,W,2014,5/8/2014 0:00,8,7,East of England,Cambridgeshire,A14,3,A1303,A1303,52.21335948,0.267274407,6.8,0,10,2223,4,339,53,2,8,2024-07-15T18:09:56.809Z,10,2639,2024-07-15T20:36:04.239Z
37086,58073,W,2014,5/8/2014 0:00,9,7,East of England,Cambridgeshire,A14,3,A1303,A1303,52.21335948,0.267274407,6.8,0,4,1221,3,223,45,6,7,2024-07-15T18:09:56.809Z,13,1509,2024-07-15T20:36:04.239Z
37087,58073,W,2014,5/8/2014 0:00,10,7,East of England,Cambridgeshire,A14,3,A1303,A1303,52.21335948,0.267274407,6.8,0,3,876,3,156,48,9,9,2024-07-15T18:09:56.809Z,18,1104,2024-07-15T20:36:04.239Z
37088,58073,W,2014,5/8/2014 0:00,11,7,East of England,Cambridgeshire,A14,3,A1303,A1303,52.21335948,0.267274407,6.8,0,1,889,1,196,40,5,6,2024-07-15T18:09:56.809Z,11,1138,2024-07-15T20:36:04.239Z
37089,58073,W,2014,5/8/2014 0:00,12,7,East of England,Cambridgeshire,A14,3,A1303,A1303,52.21335948,0.267274407,6.8,0,2,796,1,192,46,8,6,2024-07-15T18:09:56.809Z,14,1051,2024-07-15T20:36:04.239Z
